In [1]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from ast import literal_eval
import heapq
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [2]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

In [3]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [128]:
CF  = pd.read_csv('CareersFutureCleaned.csv')

In [129]:
CF

,Unnamed: 0,job_id,Job Description,Job Other Requirements,Skills,Industry,Job Title,Job Experience Required (years),Job Monthly Min Sal,Job Monthly Max Sal,Post Date
0,0,22f1bcc8d20911b85db5387f5038ff8a,The Senior Restaurant Supervisor will be resp...,NaN,"['Claims Handling', 'Customer Relations', 'Emp...","['F&B', 'General Management']",Senior Restaurant Supervisor,4,4500,7000,15/09/2020
1,1,698c038b83040c2c474bc41c6ea3d0cb,ECHOL TECH – A Team of Technology Architect a...,NaN,"['Artificial Intelligence', 'Big Data', 'C++',...",['Information Technology'],Data Scientist and DevSecOps Developer,3,5000,7000,15/09/2020
2,2,b43fe59893b7348a94fac355d66a4391,Position Office Admin Support Administration ...,NaN,"['Data', 'MS Office', 'MS Office Suite', 'MS O...",['Education and Training'],Admin Support (School),1,1100,1500,15/09/2020
3,3,8a6748454b0adee2644c207a80a03fc6,Lead engineer for Techical Requirements Analy...,NaN,"['Analysis', 'Java', 'Java Application Develop...",['Information Technology'],Technical Lead/ Senior Consultant/ Senior Tech...,5,5000,9000,15/09/2020
4,4,f121bbb503160bfc4faae799b70dee8d,We are seeking individuals that are driven dy...,NaN,"['Contract Recruitment', 'Coordination', 'Exec...","['Human Resources', 'Sales / Retail']",Senior Recruitment Consultant,2,2800,4600,15/09/2020
...,...,...,...,...,...,...,...,...,...,...,...
39495,39495,5b221d4e745cc8429cf2469f0f484154,The People Organization Group POG is the GovT...,NaN,"['Agile Methodologies', 'Algorithms', 'Artific...","['Information Technology', 'Public / Civil Ser...",Data Scientist (Human Capital),0,5000,8000,27/03/2020
39496,39496,7ceef062786a69a3e8ee2b04d5fa80bb,GovTech builds key digital platforms and infr...,NaN,"['Business Analysis', 'Business Intelligence',...","['Information Technology', 'Public / Civil Ser...","Engagement Manager, eKYC",0,5000,8000,27/03/2020
39497,39497,1d72acd8196ffbab05f6375442ad8322,Imagine citizens having a common and secure d...,NaN,"['Business Analysis', 'Business Development', ...","['Information Technology', 'Public / Civil Ser...",Security Operations Manager/Lead (National Dig...,0,6000,9000,27/03/2020
39498,39498,f2a6a6acc9f0bdec2e88359439556bab,As a Data Engineer in GovTech’s Data Science ...,NaN,"['Agile Methodologies', 'Cloud Computing', 'Di...","['Engineering', 'Public / Civil Service']",Data Engineering (Visualisation),0,5000,9000,27/03/2020


In [130]:
CF['clean_jd'] = CF.apply(lambda x: str(x['Job Description']), axis=1)
CF['clean_jd'] = CF['clean_jd'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
CF['clean_jd'] = CF["clean_jd"].fillna('').apply(lambda x: x.lower())
CF['Skills'] = CF['Skills'].apply(lambda x: eval(x))
CF['Skills'] = CF["Skills"].apply(lambda a: [x.lower() for x in a])
CF['Industry'] = CF['Industry'].apply(lambda x: eval(x))

In [131]:
CF_jd_skills = CF[['clean_jd', 'Industry', 'Job Title', 'Skills']]

In [132]:
CF_jd_skills

,clean_jd,Industry,Job Title,Skills
0,the senior restaurant supervisor will be resp...,"[F&B, General Management]",Senior Restaurant Supervisor,"[claims handling, customer relations, employee..."
1,echol tech a team of technology architect at ...,[Information Technology],Data Scientist and DevSecOps Developer,"[artificial intelligence, big data, c++, commu..."
2,position office admin support administration ...,[Education and Training],Admin Support (School),"[data, ms office, ms office suite, ms office t..."
3,lead engineer for techical requirements analy...,[Information Technology],Technical Lead/ Senior Consultant/ Senior Tech...,"[analysis, java, java application development,..."
4,we are seeking individuals that are driven dy...,"[Human Resources, Sales / Retail]",Senior Recruitment Consultant,"[contract recruitment, coordination, executive..."
...,...,...,...,...
39495,the people organization group pog is the govt...,"[Information Technology, Public / Civil Service]",Data Scientist (Human Capital),"[agile methodologies, algorithms, artificial i..."
39496,govtech builds key digital platforms and infr...,"[Information Technology, Public / Civil Service]","Engagement Manager, eKYC","[business analysis, business intelligence, bus..."
39497,imagine citizens having a common and secure d...,"[Information Technology, Public / Civil Service]",Security Operations Manager/Lead (National Dig...,"[business analysis, business development, busi..."
39498,as a data engineer in govtech s data science ...,"[Engineering, Public / Civil Service]",Data Engineering (Visualisation),"[agile methodologies, cloud computing, distrib..."


In [150]:
SFW  = pd.read_csv('sfw_sector.csv')

In [151]:
SFW

,Unnamed: 0,Job_Role_Replaced,Sector,Skill description,Skill Title
0,0,(airsid operations),Air Transport,follow instructions processes accidents incide...,Accident and Incident Response Management Airp...
1,1,(baggag services),Air Transport,follow instructions processes accidents incide...,Accident and Incident Response Management Airc...
2,2,(cargo),Air Transport,follow instructions processes accidents incide...,Accident and Incident Response Management Air ...
3,3,(contact centr operations)/ (servic centres)/ ...,Air Transport,coordinate distribution channel partners provi...,Airline Distribution Channel Strategies Airlin...
4,4,(digit delivery),Built Environment,use 3d modelling software create basic threedi...,3D Modelling Applications Integration Applicat...
...,...,...,...,...,...
1244,1244,workplac safeti and health supervisor,Workplace Safety and Health,participate emergency response drills suggest ...,Emergency Response Management Hazards Identifi...
1245,1245,workshop - bus workshop,Public Transport,prepare business units operational budgets obj...,Budgeting Bus Air-Conditioning Systems Mainten...
1246,1246,workshop supervisor / foreman - bus workshop,Public Transport,conduct corrective maintenance bus airconditio...,Bus Air-Conditioning Systems Maintenance Bus B...
1247,1247,youth work associ,Social Service,support development case care plans support cl...,Case and Care Planning Casework Evaluation Cas...


In [146]:
SFW_new2

,Unnamed: 0,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,0,#NAME?,Media,"['Executive - Content Acquisition', 'Executive...",['apply negotiation skills techniques document...,"['business negotiation', 'content acquisition ..."
1,1,(airsid operations),Air Transport,['Manager (Airside Operations)'],['follow instructions processes accidents inci...,"['accident and incident response management', ..."
2,2,(baggag services),Air Transport,"['Executive (Baggage Services)', 'Manager (Bag...",['follow instructions processes accidents inci...,"['accident and incident response management', ..."
3,3,(cargo),Air Transport,"['Executive (Cargo)', 'Manager (Cargo)']",['follow instructions processes accidents inci...,"['accident and incident response management', ..."
4,4,(contact centr operations)/ (servic centres)/ ...,Air Transport,['Executive (Contact Centre Operations)/Execut...,['coordinate distribution channel partners pro...,"['airline distribution channel strategies', 'a..."
...,...,...,...,...,...,...
1234,1234,workplac safeti and health supervis,Workplace Safety and Health,['Workplace Safety and Health Supervisor'],['participate emergency response drills sugges...,"['emergency response management', 'hazards ide..."
1235,1235,workshop - bus workshop,Public Transport,['Workshop Manager - Bus Workshop'],['prepare business units operational budgets o...,"['budgeting', 'bus air-conditioning systems ma..."
1236,1236,workshop supervis / foreman - bus workshop,Public Transport,['Workshop Supervisor / Senior Foreman - Bus W...,['conduct corrective maintenance bus aircondit...,"['bus air-conditioning systems maintenance', '..."
1237,1237,youth work associ,Social Service,['Youth Work Associate'],['support development case care plans support ...,"['case and care planning', 'casework evaluatio..."


In [153]:
SFW_new2[SFW_new2['Job_Role_Replaced'].str.contains('#NAME?', na=False)]

,Unnamed: 0,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,0,#NAME?,Media,"['Executive - Content Acquisition', 'Executive...",['apply negotiation skills techniques document...,"['business negotiation', 'content acquisition ..."


In [142]:
x = np.setdiff1d((SFW['Job_Role_Replaced']).astype(str), (SFW_new2['Job_Role_Replaced']).astype(str))

In [144]:
x

array(['- linear media infrastructur', '- linear media oper', '- localis',
       '- on-demand media technolog and oper', '- product manag',
       '- standard and practic', 'airport emerg offic (specialist)',
       'assessor', 'assist', 'assist (network planning)',
       'assist (oper & maintenance/specialis systems/projects)',
       'assist /', 'assist / assist',
       'assist / engin (project development)',
       'assist charter broker/traine charter broker',
       'assist civil and structur engineer/techn (civil and structur engineering)',
       'assist content and experi develop / assist curat',
       'assist coordin and reserv', 'assist crew',
       'assist custom support /assist travel consult (custom support)',
       'assist design engin', 'assist educ and programm',
       'assist engineer/ assist engin - automat fare collect',
       'assist engineer/ assist engin - engin train',
       'assist engineer/ assist engin - mechan and electr',
       'assist engineer/ as

In [120]:
SFW[SFW['Job_Role_Replaced'].str.contains('#NAME?')]

,Unnamed: 0,Job_Role_Replaced,Sector,Skill description,Skill Title


In [110]:
SFW_new = pd.read_csv('sf_model_final.csv')

In [111]:
SFW_new

,Unnamed: 0,Job Role,Sector,Skill Title,Skill description,Skill Description,Job_Role_Replaced
0,0,Airport Emergency Assistant Manager,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg assist
1,20,Airport Emergency Manager,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg
2,39,Airport Emergency Officer,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg offic
3,1,Airport Emergency Assistant Manager,Air Transport,Airport Operations Management,identify different functions duties airport op...,"identify,different,functions,duties,airport,op...",airport emerg assist
4,21,Airport Emergency Manager,Air Transport,Airport Operations Management,identify different functions duties airport op...,"identify,different,functions,duties,airport,op...",airport emerg
...,...,...,...,...,...,...,...
31621,31621,Technical Officer,Built Environment,Technical Writing,draft technical reports according established ...,"draft,technical,reports,according,established,...",technic offic
31622,31622,Technical Officer,Built Environment,Technology Application,adopt new technologies enhance operations proc...,"adopt,new,technologies,enhance,operations,proc...",technic offic
31623,31623,Technical Officer,Built Environment,Value Engineering,apply developed processes procedures minimise ...,"apply,developed,processes,procedures,minimise,...",technic offic
31624,31624,Technical Officer,Built Environment,Workplace Safety and Health Culture Development,support workplace safety health wsh culture pr...,"support,workplace,safety,health,wsh,culture,pr...",technic offic


In [96]:
SFW_new.drop_duplicates(subset=['Job_Role_Replaced', 'Sector'])

,Unnamed: 0,Job Role,Sector,Skill Title,Skill description,Skill Description,Job_Role_Replaced
0,0,Airport Emergency Assistant Manager,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg assist
1,20,Airport Emergency Manager,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg
2,39,Airport Emergency Officer,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg offic
54,54,Airport Emergency Officer (Specialist),Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport emerg offic (specialist)
72,72,Airport Operations Centre Manager,Air Transport,Accident and Incident Response Management,follow instructions processes accidents incide...,"follow,instructions,processes,accidents_incide...",airport oper centr
...,...,...,...,...,...,...,...
31481,31481,Senior Quantity Surveyor/Senior Contracts Mana...,Built Environment,Building Information Modelling Application,apply building information modelling bim acros...,"apply,building,information,modelling,bim,acros...",quantiti surveyor/ contract / cost
31508,31508,Site Supervisor/Trade Supervisor/Project Coord...,Built Environment,Building Information Modelling Application,apply building information modelling bim acros...,"apply,building,information,modelling,bim,acros...",site supervisor/trad supervisor/project coordin
31534,31534,Specialist (Digital Delivery),Built Environment,3D Modelling,use 3d modelling software create basic threedi...,"use,3d_modelling,software,create,basic,threedi...",specialist (digit delivery)
31562,31562,Technical Executive,Built Environment,Air Conditioning and Mechanical Ventilation Sy...,carry scheduled maintenance work airconditioni...,"carry_scheduled,maintenance,work,airconditioni...",technic


In [112]:
SFW_new2 = SFW_new.groupby(['Job_Role_Replaced', 'Sector']).agg({'Job Role': lambda series: list(series), 'Skill description': lambda series: list(series), 'Skill Title':lambda series: list(series)})

In [113]:
SFW_new2

,,Job Role,Skill description,Skill Title
Job_Role_Replaced,Sector,,,
#NAME?,Media,"[Executive - Content Acquisition, Executive - ...",[apply negotiation skills techniques document ...,"[Business Negotiation, Content Acquisition Man..."
(airsid operations),Air Transport,"[Manager (Airside Operations), Manager (Airsid...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
(baggag services),Air Transport,"[Executive (Baggage Services), Executive (Bagg...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
(cargo),Air Transport,"[Executive (Cargo), Executive (Cargo), Executi...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
(contact centr operations)/ (servic centres)/ (premium passeng services),Air Transport,[Executive (Contact Centre Operations)/Executi...,[coordinate distribution channel partners prov...,"[Airline Distribution Channel Strategies, Airl..."
...,...,...,...,...
workplac safeti and health supervis,Workplace Safety and Health,"[Workplace Safety and Health Supervisor, Workp...",[participate emergency response drills suggest...,"[Emergency Response Management, Hazards Identi..."
workshop - bus workshop,Public Transport,"[Workshop Manager - Bus Workshop, Workshop Man...",[prepare business units operational budgets ob...,"[Budgeting, Bus Air-Conditioning Systems Maint..."
workshop supervis / foreman - bus workshop,Public Transport,[Workshop Supervisor / Senior Foreman - Bus Wo...,[conduct corrective maintenance bus airconditi...,"[Bus Air-Conditioning Systems Maintenance, Bus..."


In [114]:
SFW_new2.reset_index(inplace=True)

In [115]:
SFW_new2

,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,#NAME?,Media,"[Executive - Content Acquisition, Executive - ...",[apply negotiation skills techniques document ...,"[Business Negotiation, Content Acquisition Man..."
1,(airsid operations),Air Transport,"[Manager (Airside Operations), Manager (Airsid...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
2,(baggag services),Air Transport,"[Executive (Baggage Services), Executive (Bagg...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
3,(cargo),Air Transport,"[Executive (Cargo), Executive (Cargo), Executi...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
4,(contact centr operations)/ (servic centres)/ ...,Air Transport,[Executive (Contact Centre Operations)/Executi...,[coordinate distribution channel partners prov...,"[Airline Distribution Channel Strategies, Airl..."
...,...,...,...,...,...
1234,workplac safeti and health supervis,Workplace Safety and Health,"[Workplace Safety and Health Supervisor, Workp...",[participate emergency response drills suggest...,"[Emergency Response Management, Hazards Identi..."
1235,workshop - bus workshop,Public Transport,"[Workshop Manager - Bus Workshop, Workshop Man...",[prepare business units operational budgets ob...,"[Budgeting, Bus Air-Conditioning Systems Maint..."
1236,workshop supervis / foreman - bus workshop,Public Transport,[Workshop Supervisor / Senior Foreman - Bus Wo...,[conduct corrective maintenance bus airconditi...,"[Bus Air-Conditioning Systems Maintenance, Bus..."
1237,youth work associ,Social Service,"[Youth Work Associate, Youth Work Associate, Y...",[support development case care plans support c...,"[Case and Care Planning, Casework Evaluation, ..."


In [116]:
SFW_new2['Job Role'] = SFW_new2['Job Role'].apply(lambda x: list(dict.fromkeys(x)))

In [117]:
SFW_new2

,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,#NAME?,Media,"[Executive - Content Acquisition, Executive - ...",[apply negotiation skills techniques document ...,"[Business Negotiation, Content Acquisition Man..."
1,(airsid operations),Air Transport,[Manager (Airside Operations)],[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
2,(baggag services),Air Transport,"[Executive (Baggage Services), Manager (Baggag...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
3,(cargo),Air Transport,"[Executive (Cargo), Manager (Cargo)]",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
4,(contact centr operations)/ (servic centres)/ ...,Air Transport,[Executive (Contact Centre Operations)/Executi...,[coordinate distribution channel partners prov...,"[Airline Distribution Channel Strategies, Airl..."
...,...,...,...,...,...
1234,workplac safeti and health supervis,Workplace Safety and Health,[Workplace Safety and Health Supervisor],[participate emergency response drills suggest...,"[Emergency Response Management, Hazards Identi..."
1235,workshop - bus workshop,Public Transport,[Workshop Manager - Bus Workshop],[prepare business units operational budgets ob...,"[Budgeting, Bus Air-Conditioning Systems Maint..."
1236,workshop supervis / foreman - bus workshop,Public Transport,[Workshop Supervisor / Senior Foreman - Bus Wo...,[conduct corrective maintenance bus airconditi...,"[Bus Air-Conditioning Systems Maintenance, Bus..."
1237,youth work associ,Social Service,[Youth Work Associate],[support development case care plans support c...,"[Case and Care Planning, Casework Evaluation, ..."


In [118]:
len(SFW_new2['Job_Role_Replaced'].unique())

1182

In [121]:
SFW_new2['Job_Role_Replaced'] = SFW_new2['Job_Role_Replaced'].str.replace('head of ', '')

def clear_hierarchy(title):
    split = title.split()
    if (split[0] == 'assist') or (split[0]=='head'):
        final_title = " ".join(split[1:])
    else:
        final_title = title
    return final_title

SFW_new2['Job_Role_Replaced'] = SFW_new2['Job_Role_Replaced'].apply(lambda x: clear_hierarchy(x))
SFW_new2['Job_Role_Replaced'] = SFW_new2['Job_Role_Replaced'].str.replace('(specialist)', '')
SFW_new2['Job_Role_Replaced'] = SFW_new2['Job_Role_Replaced'].str.replace('specialist', '')

/Users/eramalam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':


In [122]:
SFW_new2.drop_duplicates(subset=['Job_Role_Replaced', 'Sector'])

,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,#NAME?,Media,"[Executive - Content Acquisition, Executive - ...",[apply negotiation skills techniques document ...,"[Business Negotiation, Content Acquisition Man..."
1,(airsid operations),Air Transport,[Manager (Airside Operations)],[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
2,(baggag services),Air Transport,"[Executive (Baggage Services), Manager (Baggag...",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
3,(cargo),Air Transport,"[Executive (Cargo), Manager (Cargo)]",[follow instructions processes accidents incid...,"[Accident and Incident Response Management, Ai..."
4,(contact centr operations)/ (servic centres)/ ...,Air Transport,[Executive (Contact Centre Operations)/Executi...,[coordinate distribution channel partners prov...,"[Airline Distribution Channel Strategies, Airl..."
...,...,...,...,...,...
1234,workplac safeti and health supervis,Workplace Safety and Health,[Workplace Safety and Health Supervisor],[participate emergency response drills suggest...,"[Emergency Response Management, Hazards Identi..."
1235,workshop - bus workshop,Public Transport,[Workshop Manager - Bus Workshop],[prepare business units operational budgets ob...,"[Budgeting, Bus Air-Conditioning Systems Maint..."
1236,workshop supervis / foreman - bus workshop,Public Transport,[Workshop Supervisor / Senior Foreman - Bus Wo...,[conduct corrective maintenance bus airconditi...,"[Bus Air-Conditioning Systems Maintenance, Bus..."
1237,youth work associ,Social Service,[Youth Work Associate],[support development case care plans support c...,"[Case and Care Planning, Casework Evaluation, ..."


In [123]:
def to_lower(x):
    y = []
    for i in range(len(x)):
        y.append(x[i].lower())
    return y

In [124]:
SFW_new2['Skill Title'] = SFW_new2['Skill Title'].apply(lambda x: to_lower(x))

In [125]:
SFW_new2

,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,#NAME?,Media,"[Executive - Content Acquisition, Executive - ...",[apply negotiation skills techniques document ...,"[business negotiation, content acquisition man..."
1,(airsid operations),Air Transport,[Manager (Airside Operations)],[follow instructions processes accidents incid...,"[accident and incident response management, ai..."
2,(baggag services),Air Transport,"[Executive (Baggage Services), Manager (Baggag...",[follow instructions processes accidents incid...,"[accident and incident response management, ai..."
3,(cargo),Air Transport,"[Executive (Cargo), Manager (Cargo)]",[follow instructions processes accidents incid...,"[accident and incident response management, ai..."
4,(contact centr operations)/ (servic centres)/ ...,Air Transport,[Executive (Contact Centre Operations)/Executi...,[coordinate distribution channel partners prov...,"[airline distribution channel strategies, airl..."
...,...,...,...,...,...
1234,workplac safeti and health supervis,Workplace Safety and Health,[Workplace Safety and Health Supervisor],[participate emergency response drills suggest...,"[emergency response management, hazards identi..."
1235,workshop - bus workshop,Public Transport,[Workshop Manager - Bus Workshop],[prepare business units operational budgets ob...,"[budgeting, bus air-conditioning systems maint..."
1236,workshop supervis / foreman - bus workshop,Public Transport,[Workshop Supervisor / Senior Foreman - Bus Wo...,[conduct corrective maintenance bus airconditi...,"[bus air-conditioning systems maintenance, bus..."
1237,youth work associ,Social Service,[Youth Work Associate],[support development case care plans support c...,"[case and care planning, casework evaluation, ..."


In [126]:
SFW_new2.to_csv('final_new_sfw.csv')

In [127]:
SFW_new2 = pd.read_csv('final_new_sfw.csv')

In [53]:
skills_lib  = pd.read_csv('tableau no jobstreet 150321.csv')

In [54]:
skills_lib

,Sector,Job Role,Skill Title,Skill Title V2,MyCFW_Skill,Score,Skill Length,max
0,Air Transport,Airport Emergency Assistant Manager,Accident and Incident Response Management,incident response management,accident and incident response management,1.000000,3,3
1,Air Transport,Airport Emergency Assistant Manager,Accident and Incident Response Management,incident response management,information risk management,0.948748,3,3
2,Air Transport,Airport Emergency Assistant Manager,Accident and Incident Response Management,incident response management,domain controller,0.948241,3,3
3,Air Transport,Airport Emergency Assistant Manager,Accident and Incident Response Management,incident response management,forensic investigation,0.945919,3,3
4,Air Transport,Airport Emergency Assistant Manager,Accident and Incident Response Management,incident response management,solution deployment,0.944941,3,3
...,...,...,...,...,...,...,...,...
87104,Built Environment,Factory Manager,Integrated Digital Delivery Application,integrated application,artificial intelligence development,0.943549,2,3
87105,Built Environment,Factory Manager,Integrated Digital Delivery Application,integrated application,trading portfolio,0.940387,2,3
87106,Built Environment,Factory Manager,Integrated Digital Delivery Application,integrated application,simulation analysis,0.938928,2,3
87107,Built Environment,Factory Manager,Integrated Digital Delivery Application,integrated application,portfolio optimisation,0.938305,2,3


In [55]:
skills_cf = skills_lib

In [56]:
skills_cf_new = skills_cf.sort_values('Score', ascending=False).drop_duplicates(['MyCFW_Skill'])

In [57]:
skills_cf_new

,Sector,Job Role,Skill Title,Skill Title V2,MyCFW_Skill,Score,Skill Length,max
0,Air Transport,Airport Emergency Assistant Manager,Accident and Incident Response Management,incident response management,accident and incident response management,1.000000,3,3
55198,Logistics,Lifting Supervisor,Material Handling Equipment (MHE) Handling,material handling,material handling equipment (mhe) handling,1.000000,2,3
7073,Environmental Services,"Environment, Health and Safety Manager",EMS Framework development and Implementation,ems development,ems framework development and implementation,1.000000,2,3
76307,Healthcare,Patient Service Assistant Supervisor,Clinical Support for Patient Service Associates,patient service associates,clinical support for patient service associates,1.000000,3,3
20757,Energy and Power,Gas Transmission and Distribution Department M...,Gas Network Operations Management,network operations management,gas network operations management,1.000000,3,3
...,...,...,...,...,...,...,...,...
22950,Financial Services,Economist,Valuation,valuation,evaluating,0.550475,1,1
10589,Logistics,Business Process Excellence Engineer/Operation...,Process Improvement and Optimisation\n\n,process improvement,process mapping,0.550095,2,2
24907,Healthcare,Pharmacy Technician Executive (Pharmacy IT and...,Pharmacy Information Technology Management,information technology,systems analysis and design,0.539768,2,3
76797,Healthcare,Pharmacy Technician Executive (Pharmacy IT and...,Pharmacy Information Technology Management,information technology,system migration,0.536201,2,3


In [58]:
CF_jd_skills

,clean_jd,Industry,Job Title,Skills
0,the senior restaurant supervisor will be resp...,"[F&B, General Management]",Senior Restaurant Supervisor,"[claims handling, customer relations, employee..."
1,echol tech a team of technology architect at ...,[Information Technology],Data Scientist and DevSecOps Developer,"[artificial intelligence, big data, c++, commu..."
2,position office admin support administration ...,[Education and Training],Admin Support (School),"[data, ms office, ms office suite, ms office t..."
3,lead engineer for techical requirements analy...,[Information Technology],Technical Lead/ Senior Consultant/ Senior Tech...,"[analysis, java, java application development,..."
4,we are seeking individuals that are driven dy...,"[Human Resources, Sales / Retail]",Senior Recruitment Consultant,"[contract recruitment, coordination, executive..."
...,...,...,...,...
39495,the people organization group pog is the govt...,"[Information Technology, Public / Civil Service]",Data Scientist (Human Capital),"[agile methodologies, algorithms, artificial i..."
39496,govtech builds key digital platforms and infr...,"[Information Technology, Public / Civil Service]","Engagement Manager, eKYC","[business analysis, business intelligence, bus..."
39497,imagine citizens having a common and secure d...,"[Information Technology, Public / Civil Service]",Security Operations Manager/Lead (National Dig...,"[business analysis, business development, busi..."
39498,as a data engineer in govtech s data science ...,"[Engineering, Public / Civil Service]",Data Engineering (Visualisation),"[agile methodologies, cloud computing, distrib..."


In [59]:
mydict = dict(zip(skills_cf_new['MyCFW_Skill'], skills_cf_new['Skill Title V2'])) # key is cf skills, value is sfw skills

In [60]:
mydict

{'accident and incident response management': 'incident response management',
 'material handling equipment (mhe) handling': 'material handling',
 'ems framework development and implementation': 'ems development',
 'clinical support for patient service associates': 'patient service associates',
 'gas network operations management': 'network operations management',
 'gas network planning': 'network planning',
 'network system planning and design': 'network system',
 'fuel terminal operations management': 'operations management',
 'vertical surface maintenance': 'vertical surface',
 'wsh performance management': 'wsh management',
 'gas meter installation and commissioning': 'gas installation',
 'contract preparation, evaluation, negotiation and tendering\n\n': 'contract preparation',
 'gas network system management': 'network management',
 'gas metering data management': 'data management',
 'wholesale transaction management': 'transaction management',
 'trading reporting and assessment':

In [61]:
np.save('skills_dict.npy', mydict) 

In [99]:
def checker(x):
    new_list = []
    for i in range(0, len(x)):
        if x[i] in mydict.keys():
            new_list.append(mydict.get(x[i]))
        else:
            new_list.append(x[i])
    return new_list

In [100]:
CF_jd_skills['skills_replaced'] = CF_jd_skills['Skills'].apply(lambda x: checker(x))

/Users/eramalam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
CF_jd_skills

,clean_jd,Industry,Job Title,Skills,skills_replaced
0,the senior restaurant supervisor will be resp...,"[F&B, General Management]",Senior Restaurant Supervisor,"[claims handling, customer relations, employee...","[claims handling, customer retention, employee..."
1,echol tech a team of technology architect at ...,[Information Technology],Data Scientist and DevSecOps Developer,"[artificial intelligence, big data, c++, commu...","[computer vision, big data, c++, communication..."
2,position office admin support administration ...,[Education and Training],Admin Support (School),"[data, ms office, ms office suite, ms office t...","[data, ms office, ms office suite, ms office t..."
3,lead engineer for techical requirements analy...,[Information Technology],Technical Lead/ Senior Consultant/ Senior Tech...,"[analysis, java, java application development,...","[analysis, java, java application development,..."
4,we are seeking individuals that are driven dy...,"[Human Resources, Sales / Retail]",Senior Recruitment Consultant,"[contract recruitment, coordination, executive...","[contract recruitment, coordination, executive..."
...,...,...,...,...,...
39495,the people organization group pog is the govt...,"[Information Technology, Public / Civil Service]",Data Scientist (Human Capital),"[agile methodologies, algorithms, artificial i...","[agile methodologies, algorithms, computer vis..."
39496,govtech builds key digital platforms and infr...,"[Information Technology, Public / Civil Service]","Engagement Manager, eKYC","[business analysis, business intelligence, bus...","[business analysis, business analytics, busine..."
39497,imagine citizens having a common and secure d...,"[Information Technology, Public / Civil Service]",Security Operations Manager/Lead (National Dig...,"[business analysis, business development, busi...","[business analysis, business development, busi..."
39498,as a data engineer in govtech s data science ...,"[Engineering, Public / Civil Service]",Data Engineering (Visualisation),"[agile methodologies, cloud computing, distrib...","[agile methodologies, cloud computing, distrib..."


In [102]:
#only 1 industry
CF_jd_skills_1_ind = CF_jd_skills.loc[np.array(list(map(len, CF_jd_skills['Industry'].values)))==1]                                 

In [103]:
CF_jd_skills_1_ind['Industry_nolist'] = CF_jd_skills_1_ind['Industry'].apply(lambda x: ''.join(x))

/Users/eramalam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [104]:
CF_jd_skills_1_ind

,clean_jd,Industry,Job Title,Skills,skills_replaced,Industry_nolist
1,echol tech a team of technology architect at ...,[Information Technology],Data Scientist and DevSecOps Developer,"[artificial intelligence, big data, c++, commu...","[computer vision, big data, c++, communication...",Information Technology
2,position office admin support administration ...,[Education and Training],Admin Support (School),"[data, ms office, ms office suite, ms office t...","[data, ms office, ms office suite, ms office t...",Education and Training
3,lead engineer for techical requirements analy...,[Information Technology],Technical Lead/ Senior Consultant/ Senior Tech...,"[analysis, java, java application development,...","[analysis, java, java application development,...",Information Technology
5,performs metallurgical sample preparation cut...,[Manufacturing],Quality Inspector/Met Lab Technician,"[analysis, calibration, data, inspection, main...","[analysis, calibration, data, inspection, main...",Manufacturing
6,job scope coordinate and perform secretariat ...,[Information Technology],SAP Consultant (HCM),"[abap, business process, databases, erp, hris,...","[abap, business process, databases, erp, emplo...",Information Technology
...,...,...,...,...,...,...
39477,scdf the lifesaving force we are part of sing...,[Public / Civil Service],SCDF Officer (Direct-Entry Lieutenant),"[cpr certified, customer service, disaster res...","[cpr certified, customer service, disaster res...",Public / Civil Service
39484,overview foreign service administration speci...,[Public / Civil Service],Foreign Service Administration Specialist,"[analysis, budgets, business strategy, custome...","[analysis, budgets, business strategy, custome...",Public / Civil Service
39485,job scope drive digitisation of the ministrys...,[Public / Civil Service],Assistant Director/Information Services,"[axure, e-commerce, experience design, heurist...","[digital production, e-commerce, experience de...",Public / Civil Service
39486,job scope as deputy directorperformance manag...,[Public / Civil Service],Deputy Director/Performance Management,"[change management, deferred compensation, emp...","[change management, employee relations, employ...",Public / Civil Service


In [4]:
CF_jd_skills_1_ind = pd.read_csv('single_ind_cf.csv')
CF_jd_skills_1_ind

,Unnamed: 0,clean_jd,Industry,Job Title,Skills,skills_replaced,Industry_nolist
0,1,echol tech a team of technology architect at ...,['Information Technology'],Data Scientist and DevSecOps Developer,"['artificial intelligence', 'big data', 'c++',...","['computer vision', 'big data', 'c++', 'commun...",Information Technology
1,2,position office admin support administration ...,['Education and Training'],Admin Support (School),"['data', 'ms office', 'ms office suite', 'ms o...","['data', 'ms office', 'ms office suite', 'ms o...",Education and Training
2,3,lead engineer for techical requirements analy...,['Information Technology'],Technical Lead/ Senior Consultant/ Senior Tech...,"['analysis', 'java', 'java application develop...","['analysis', 'java', 'java application develop...",Information Technology
3,5,performs metallurgical sample preparation cut...,['Manufacturing'],Quality Inspector/Met Lab Technician,"['analysis', 'calibration', 'data', 'inspectio...","['analysis', 'calibration', 'data', 'inspectio...",Manufacturing
4,6,job scope coordinate and perform secretariat ...,['Information Technology'],SAP Consultant (HCM),"['abap', 'business process', 'databases', 'erp...","['abap', 'business process', 'databases', 'erp...",Information Technology
...,...,...,...,...,...,...,...
27144,39477,scdf the lifesaving force we are part of sing...,['Public / Civil Service'],SCDF Officer (Direct-Entry Lieutenant),"['cpr certified', 'customer service', 'disaste...","['cpr certified', 'customer service', 'disaste...",Public / Civil Service
27145,39484,overview foreign service administration speci...,['Public / Civil Service'],Foreign Service Administration Specialist,"['analysis', 'budgets', 'business strategy', '...","['analysis', 'budgets', 'business strategy', '...",Public / Civil Service
27146,39485,job scope drive digitisation of the ministrys...,['Public / Civil Service'],Assistant Director/Information Services,"['axure', 'e-commerce', 'experience design', '...","['digital production', 'e-commerce', 'experien...",Public / Civil Service
27147,39486,job scope as deputy directorperformance manag...,['Public / Civil Service'],Deputy Director/Performance Management,"['change management', 'deferred compensation',...","['change management', 'employee relations', 'e...",Public / Civil Service


In [6]:
CF_jd_skills_1_ind.iloc[125]

Unnamed: 0                                                       162
clean_jd            reports to project manager coordinate all me ...
Industry                               ['Building and Construction']
Job Title               Mechanical & Electrical Supervisor/ Engineer
Skills             ['continuous improvement', 'electrical enginee...
skills_replaced    ['continuous improvement', 'electronic enginee...
Industry_nolist                            Building and Construction
Name: 125, dtype: object

In [5]:
SFW = pd.read_csv('final_new_sfw.csv')
SFW

,Unnamed: 0,Job_Role_Replaced,Sector,Job Role,Skill description,Skill Title
0,0,#NAME?,Media,"['Executive - Content Acquisition', 'Executive...",['apply negotiation skills techniques document...,"['Business Negotiation', 'Content Acquisition ..."
1,1,(airsid operations),Air Transport,"['Manager (Airside Operations)', 'Manager (Air...",['follow instructions processes accidents inci...,"['Accident and Incident Response Management', ..."
2,2,(baggag services),Air Transport,"['Executive (Baggage Services)', 'Executive (B...",['follow instructions processes accidents inci...,"['Accident and Incident Response Management', ..."
3,3,(cargo),Air Transport,"['Executive (Cargo)', 'Executive (Cargo)', 'Ex...",['follow instructions processes accidents inci...,"['Accident and Incident Response Management', ..."
4,4,(contact centr operations)/ (servic centres)/ ...,Air Transport,['Executive (Contact Centre Operations)/Execut...,['coordinate distribution channel partners pro...,"['Airline Distribution Channel Strategies', 'A..."
...,...,...,...,...,...,...
1234,1234,workplac safeti and health supervis,Workplace Safety and Health,"['Workplace Safety and Health Supervisor', 'Wo...",['participate emergency response drills sugges...,"['Emergency Response Management', 'Hazards Ide..."
1235,1235,workshop - bus workshop,Public Transport,"['Workshop Manager - Bus Workshop', 'Workshop ...",['prepare business units operational budgets o...,"['Budgeting', 'Bus Air-Conditioning Systems Ma..."
1236,1236,workshop supervis / foreman - bus workshop,Public Transport,['Workshop Supervisor / Senior Foreman - Bus W...,['conduct corrective maintenance bus aircondit...,"['Bus Air-Conditioning Systems Maintenance', '..."
1237,1237,youth work associ,Social Service,"['Youth Work Associate', 'Youth Work Associate...",['support development case care plans support ...,"['Case and Care Planning', 'Casework Evaluatio..."


In [218]:
CF_jd_skills_1_ind['Industry_nolist'].unique()

array(['Information Technology', 'Education and Training',
       'Manufacturing', 'Customer Service',
       'Accounting / Auditing / Taxation', 'F&B',
       'Sciences / Laboratory / R&D', 'Admin / Secretarial',
       'Sales / Retail', 'Others', 'Engineering',
       'Marketing / Public Relations', 'Logistics / Supply Chain',
       'Banking and Finance', 'General Work', 'Human Resources',
       'Advertising / Media', 'Design', 'Building and Construction',
       'General Management', 'Telecommunications',
       'Personal Care / Beauty', 'Repair and Maintenance', 'Insurance',
       'Social Services', 'Healthcare / Pharmaceutical',
       'Real Estate / Property Management', 'Security and Investigation',
       'Consulting', 'Environment / Health',
       'Architecture / Interior Design', 'Legal', 'Entertainment',
       'Medical / Therapy Services', 'Purchasing / Merchandising',
       'Public / Civil Service', 'Professional Services', 'Hospitality',
       'Risk Management', 'Ev

In [ ]:
# not done:

# Manufacturing, Cust Serv, F&B, Sci, Admin, Others, Market, GW, Building, 
# GM, Tele, Repair, Social, Real Est, Sec, Legal, Entert, Purchasing, Public & Civil, Prof Serv, Hosp, Events

# early 5: aerospace, food serv, hotel, prec engine, early child

In [219]:
skills_cf_new['Sector'].unique()

array(['Air Transport', 'Logistics', 'Environmental Services',
       'Healthcare', 'Energy and Power', 'Marine & Offshore',
       'Energy & Chemicals', 'Financial Services', 'Engineering Services',
       'Food Manufacturing', 'Public Transport', 'Design', 'Media',
       'Landscape', 'Social Service', 'Training and Adult Education',
       'Intellectual Property', 'Human Resource', 'Tourism', 'Retail',
       'Security', 'Built Environment', 'Sea Transport', 'BioPharm',
       'Workplace Safety and Health', 'Wholesale Trade', 'Accountancy',
       'Infocomm Technology', 'Electronics'], dtype=object)

In [ ]:
def replace_job_portal_skills(df):
    
    df['skills_replaced'] = df['Skills'].apply(lambda x: checker(x))
    
    def checker(x):
        new_list = []
        for i in range(0, len(x)):
            if x[i] in mydict.keys():
                new_list.append(mydict.get(x[i]))
            else:
                new_list.append(x[i])
        return new_list
    
    return df

In [29]:
def similar_sfw_jobs(jp_industries, sfw_industries, job_portal_df, sfw_df):
    
    industries_list = []
    for i in jp_industries:
        industries_list.append([i])
    
    industries_dataframes = []
    
    for i in industries_list:
        df = job_portal_df.Industry.apply(lambda x: any(item for item in i if item in x))
        df = job_portal_df[df]
        df.reset_index(inplace=True)
        industries_dataframes.append(df)

    industries_df = pd.concat(industries_dataframes)
    industries_df.reset_index(inplace=True)

    sfw_sector_industries = sfw_df[sfw_df['Sector'] == sfw_industries]
    sfw_sector_industries.reset_index(inplace=True)

    print("outer loop will run ", len(industries_df), " times")
    print("inner loop will run ", len(sfw_sector_industries), " times")

    scores_main = []
    sfw_jobs_main = []

    for i in range(0, len(industries_df)):
        scores = []
        sfw_jobs = []
        jp = nlp(str(industries_df.iloc[i]['skills_replaced']))

        for j in range(0, len(sfw_sector_industries)):
            sfw = nlp(sfw_sector_industries['Skill Title'][j])
            scores.append(sfw.similarity(jp))
            sfw_jobs.append(sfw_sector_industries['Job Role'][j])

        scores_main.append(scores)
        sfw_jobs_main.append(sfw_jobs)
        print("done job portal job role no.: " + str(i))
    
    closest_1_sfw_jobs = []
    closest_1_score = []

    closest_2_sfw_jobs = []
    closest_2_score = []

    closest_3_sfw_jobs = []
    closest_3_score = []
    jp_jobs = []

    for i in range(len(scores_main)):
        ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
        closest_1_sfw_jobs.append(sfw_sector_industries['Job Role'][ind1])
        closest_1_score.append(scores_main[i][ind1])

        closest_2_sfw_jobs.append(sfw_sector_industries['Job Role'][ind2])
        closest_2_score.append(scores_main[i][ind2])

        closest_3_sfw_jobs.append(sfw_sector_industries['Job Role'][ind3])
        closest_3_score.append(scores_main[i][ind3])
        jp_jobs.append(industries_df['Job Title'][i])

    df_industries = pd.DataFrame({'job portal_jobs':jp_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_2_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_3_score})
    
    return df_industries

In [ ]:
fin_serv = ['Banking and Finance', 'Insurance', 'Consulting', 'Risk Management']
fin_serv_df = similar_sfw_jobs(fin_serv, 'Financial Services', CF_jd_skills_1_ind, SFW)

In [30]:
IT = ['Information Technology']
IT_df = similar_sfw_jobs(IT, 'Infocomm Technology', CF_jd_skills_1_ind, SFW)

outer loop will run  6359  times
outer loop will run  86  times
done job portal job role no.: 0
done job portal job role no.: 1
done job portal job role no.: 2
done job portal job role no.: 3
done job portal job role no.: 4
done job portal job role no.: 5
done job portal job role no.: 6
done job portal job role no.: 7
done job portal job role no.: 8
done job portal job role no.: 9
done job portal job role no.: 10
done job portal job role no.: 11
done job portal job role no.: 12
done job portal job role no.: 13
done job portal job role no.: 14
done job portal job role no.: 15
done job portal job role no.: 16
done job portal job role no.: 17
done job portal job role no.: 18
done job portal job role no.: 19
done job portal job role no.: 20
done job portal job role no.: 21
done job portal job role no.: 22
done job portal job role no.: 23
done job portal job role no.: 24
done job portal job role no.: 25
done job portal job role no.: 26
done job portal job role no.: 27
done job portal job ro

done job portal job role no.: 243
done job portal job role no.: 244
done job portal job role no.: 245
done job portal job role no.: 246
done job portal job role no.: 247
done job portal job role no.: 248
done job portal job role no.: 249
done job portal job role no.: 250
done job portal job role no.: 251
done job portal job role no.: 252
done job portal job role no.: 253
done job portal job role no.: 254
done job portal job role no.: 255
done job portal job role no.: 256
done job portal job role no.: 257
done job portal job role no.: 258
done job portal job role no.: 259
done job portal job role no.: 260
done job portal job role no.: 261
done job portal job role no.: 262
done job portal job role no.: 263
done job portal job role no.: 264
done job portal job role no.: 265
done job portal job role no.: 266
done job portal job role no.: 267
done job portal job role no.: 268
done job portal job role no.: 269
done job portal job role no.: 270
done job portal job role no.: 271
done job porta

done job portal job role no.: 484
done job portal job role no.: 485
done job portal job role no.: 486
done job portal job role no.: 487
done job portal job role no.: 488
done job portal job role no.: 489
done job portal job role no.: 490
done job portal job role no.: 491
done job portal job role no.: 492
done job portal job role no.: 493
done job portal job role no.: 494
done job portal job role no.: 495
done job portal job role no.: 496
done job portal job role no.: 497
done job portal job role no.: 498
done job portal job role no.: 499
done job portal job role no.: 500
done job portal job role no.: 501
done job portal job role no.: 502
done job portal job role no.: 503
done job portal job role no.: 504
done job portal job role no.: 505
done job portal job role no.: 506
done job portal job role no.: 507
done job portal job role no.: 508
done job portal job role no.: 509
done job portal job role no.: 510
done job portal job role no.: 511
done job portal job role no.: 512
done job porta

done job portal job role no.: 725
done job portal job role no.: 726
done job portal job role no.: 727
done job portal job role no.: 728
done job portal job role no.: 729
done job portal job role no.: 730
done job portal job role no.: 731
done job portal job role no.: 732
done job portal job role no.: 733
done job portal job role no.: 734
done job portal job role no.: 735
done job portal job role no.: 736
done job portal job role no.: 737
done job portal job role no.: 738
done job portal job role no.: 739
done job portal job role no.: 740
done job portal job role no.: 741
done job portal job role no.: 742
done job portal job role no.: 743
done job portal job role no.: 744
done job portal job role no.: 745
done job portal job role no.: 746
done job portal job role no.: 747
done job portal job role no.: 748
done job portal job role no.: 749
done job portal job role no.: 750
done job portal job role no.: 751
done job portal job role no.: 752
done job portal job role no.: 753
done job porta

done job portal job role no.: 966
done job portal job role no.: 967
done job portal job role no.: 968
done job portal job role no.: 969
done job portal job role no.: 970
done job portal job role no.: 971
done job portal job role no.: 972
done job portal job role no.: 973
done job portal job role no.: 974
done job portal job role no.: 975
done job portal job role no.: 976
done job portal job role no.: 977
done job portal job role no.: 978
done job portal job role no.: 979
done job portal job role no.: 980
done job portal job role no.: 981
done job portal job role no.: 982
done job portal job role no.: 983
done job portal job role no.: 984
done job portal job role no.: 985
done job portal job role no.: 986
done job portal job role no.: 987
done job portal job role no.: 988
done job portal job role no.: 989
done job portal job role no.: 990
done job portal job role no.: 991
done job portal job role no.: 992
done job portal job role no.: 993
done job portal job role no.: 994
done job porta

done job portal job role no.: 1202
done job portal job role no.: 1203
done job portal job role no.: 1204
done job portal job role no.: 1205
done job portal job role no.: 1206
done job portal job role no.: 1207
done job portal job role no.: 1208
done job portal job role no.: 1209
done job portal job role no.: 1210
done job portal job role no.: 1211
done job portal job role no.: 1212
done job portal job role no.: 1213
done job portal job role no.: 1214
done job portal job role no.: 1215
done job portal job role no.: 1216
done job portal job role no.: 1217
done job portal job role no.: 1218
done job portal job role no.: 1219
done job portal job role no.: 1220
done job portal job role no.: 1221
done job portal job role no.: 1222
done job portal job role no.: 1223
done job portal job role no.: 1224
done job portal job role no.: 1225
done job portal job role no.: 1226
done job portal job role no.: 1227
done job portal job role no.: 1228
done job portal job role no.: 1229
done job portal job 

KeyboardInterrupt: 

In [33]:
design = ['Architecture / Interior Design', 'Design']
design_df = similar_sfw_jobs(design, 'Design', CF_jd_skills_1_ind, SFW)
design_df.to_csv('cfw_design.csv')

outer loop will run  648  times
outer loop will run  23  times
done job portal job role no.: 0
done job portal job role no.: 1
done job portal job role no.: 2
done job portal job role no.: 3
done job portal job role no.: 4
done job portal job role no.: 5
done job portal job role no.: 6
done job portal job role no.: 7
done job portal job role no.: 8
done job portal job role no.: 9
done job portal job role no.: 10
done job portal job role no.: 11
done job portal job role no.: 12
done job portal job role no.: 13
done job portal job role no.: 14
done job portal job role no.: 15
done job portal job role no.: 16
done job portal job role no.: 17
done job portal job role no.: 18
done job portal job role no.: 19
done job portal job role no.: 20
done job portal job role no.: 21
done job portal job role no.: 22
done job portal job role no.: 23
done job portal job role no.: 24
done job portal job role no.: 25
done job portal job role no.: 26
done job portal job role no.: 27
done job portal job rol

done job portal job role no.: 243
done job portal job role no.: 244
done job portal job role no.: 245
done job portal job role no.: 246
done job portal job role no.: 247
done job portal job role no.: 248
done job portal job role no.: 249
done job portal job role no.: 250
done job portal job role no.: 251
done job portal job role no.: 252
done job portal job role no.: 253
done job portal job role no.: 254
done job portal job role no.: 255
done job portal job role no.: 256
done job portal job role no.: 257
done job portal job role no.: 258
done job portal job role no.: 259
done job portal job role no.: 260
done job portal job role no.: 261
done job portal job role no.: 262
done job portal job role no.: 263
done job portal job role no.: 264
done job portal job role no.: 265
done job portal job role no.: 266
done job portal job role no.: 267
done job portal job role no.: 268
done job portal job role no.: 269
done job portal job role no.: 270
done job portal job role no.: 271
done job porta

done job portal job role no.: 484
done job portal job role no.: 485
done job portal job role no.: 486
done job portal job role no.: 487
done job portal job role no.: 488
done job portal job role no.: 489
done job portal job role no.: 490
done job portal job role no.: 491
done job portal job role no.: 492
done job portal job role no.: 493
done job portal job role no.: 494
done job portal job role no.: 495
done job portal job role no.: 496
done job portal job role no.: 497
done job portal job role no.: 498
done job portal job role no.: 499
done job portal job role no.: 500
done job portal job role no.: 501
done job portal job role no.: 502
done job portal job role no.: 503
done job portal job role no.: 504
done job portal job role no.: 505
done job portal job role no.: 506
done job portal job role no.: 507
done job portal job role no.: 508
done job portal job role no.: 509
done job portal job role no.: 510
done job portal job role no.: 511
done job portal job role no.: 512
done job porta

In [31]:
hr = ['Human Resources']
hr_df = similar_sfw_jobs(hr, 'Human Resource', CF_jd_skills_1_ind, SFW)
hr_df.to_csv('cfw_human_resource.csv')

outer loop will run  456  times
outer loop will run  20  times
done job portal job role no.: 0
done job portal job role no.: 1
done job portal job role no.: 2
done job portal job role no.: 3
done job portal job role no.: 4
done job portal job role no.: 5
done job portal job role no.: 6
done job portal job role no.: 7
done job portal job role no.: 8
done job portal job role no.: 9
done job portal job role no.: 10
done job portal job role no.: 11
done job portal job role no.: 12
done job portal job role no.: 13
done job portal job role no.: 14
done job portal job role no.: 15
done job portal job role no.: 16
done job portal job role no.: 17
done job portal job role no.: 18
done job portal job role no.: 19
done job portal job role no.: 20
done job portal job role no.: 21
done job portal job role no.: 22
done job portal job role no.: 23
done job portal job role no.: 24
done job portal job role no.: 25
done job portal job role no.: 26
done job portal job role no.: 27
done job portal job rol

done job portal job role no.: 243
done job portal job role no.: 244
done job portal job role no.: 245
done job portal job role no.: 246
done job portal job role no.: 247
done job portal job role no.: 248
done job portal job role no.: 249
done job portal job role no.: 250
done job portal job role no.: 251
done job portal job role no.: 252
done job portal job role no.: 253
done job portal job role no.: 254
done job portal job role no.: 255
done job portal job role no.: 256
done job portal job role no.: 257
done job portal job role no.: 258
done job portal job role no.: 259
done job portal job role no.: 260
done job portal job role no.: 261
done job portal job role no.: 262
done job portal job role no.: 263
done job portal job role no.: 264
done job portal job role no.: 265
done job portal job role no.: 266
done job portal job role no.: 267
done job portal job role no.: 268
done job portal job role no.: 269
done job portal job role no.: 270
done job portal job role no.: 271
done job porta

# All industries below are done using old code:

In [23]:
acc = [['Information Technology']]
acc_dataframes = []

for i in acc:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    acc_dataframes.append(df)

acc_df = pd.concat(acc_dataframes)
acc_df.reset_index(inplace=True)

sfw_sector_acc = SFW[SFW['Sector'] == "Infocomm Technology"]
sfw_sector_acc.reset_index(inplace=True)

print(len(acc_df))
print(len(sfw_sector_acc))

6359
86


In [24]:
acc_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(126):
    scores = []
    sfw_jobs = []
    cf = nlp(str(acc_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_acc)):
        sfw = nlp(str(sfw_sector_acc['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

In [25]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_acc['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_acc['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_acc['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(acc_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_2_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_3_score})
df_acc.tail(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
76,Cloud Operations Manager,"['SysOps Engineer', 'SysOps Engineer', 'SysOps...",0.986281,"['Automation and Orchestration Engineer', 'Aut...",0.985898,"['Infrastructure Engineering Manager', 'Infras...",0.985695
77,(IT)- Application Developer,"['Quality Assurance Engineer', 'Quality Assura...",0.971403,"['DevOps Engineer', 'DevOps Engineer', 'DevOps...",0.967318,"['Quality Engineer', 'Quality Engineer', 'Qual...",0.967254
78,(IT)- Business Analyst,['Head of Data Science and Artificial Intellig...,0.989459,"['Quality Assurance Engineer', 'Quality Assura...",0.989145,"['Quality Engineer', 'Quality Engineer', 'Qual...",0.988559
79,(IT)- IOS Developer,"['Quality Assurance Engineer', 'Quality Assura...",0.940888,['Artificial Intelligence/Machine Learning Eng...,0.939167,"['Data Analyst/Associate Data Engineer', 'Data...",0.939100
80,(IT)- Android Developer,"['Associate Software Engineer', 'Associate Sof...",0.953192,['Artificial Intelligence/Machine Learning Eng...,0.951595,"['Quality Assurance Engineer', 'Quality Assura...",0.950923
81,Quality Assurance Engineer,"['Quality Assurance Engineer', 'Quality Assura...",0.978391,"['Head of Quality', 'Head of Quality', 'Head o...",0.974306,"['Quality Engineer', 'Quality Engineer', 'Qual...",0.974200
82,Program Manager,"['Head of Operations and Support', 'Head of Op...",0.990393,"['Chief Technology Officer', 'Chief Technology...",0.990137,"['Pre-Sales Director', 'Pre-Sales Director', '...",0.989914
83,Application support (L2),"['Quality Assurance Engineer', 'Quality Assura...",0.966660,"['Head of Operations and Support', 'Head of Op...",0.964301,"['Quality Assurance Manager', 'Quality Assuran...",0.963894
84,Junior Desktop Engineer #SGUnitedTraineeships,"['Quality Assurance Engineer', 'Quality Assura...",0.968352,"['Head of Quality', 'Head of Quality', 'Head o...",0.968237,"['Quality Assurance Manager', 'Quality Assuran...",0.968191
85,Desktop Engineer #SGUnitedTraineeships,"['Quality Assurance Engineer', 'Quality Assura...",0.981578,"['DevOps Engineer', 'DevOps Engineer', 'DevOps...",0.980945,['Artificial Intelligence/Machine Learning Eng...,0.980860


# Acc

In [7]:
acc = [['Accounting / Auditing / Taxation']]
acc_dataframes = []

for i in acc:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    acc_dataframes.append(df)

acc_df = pd.concat(acc_dataframes)
acc_df.reset_index(inplace=True)

sfw_sector_acc = SFW[SFW['Sector'] == "Accountancy"]
sfw_sector_acc.reset_index(inplace=True)

In [8]:
print(len(acc_df))
print(len(sfw_sector_acc))

1300
39


In [9]:
acc_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(acc_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(acc_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_acc)):
        sfw = nlp(str(sfw_sector_acc['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

done cf job role: 378
done cf job role: 379
done cf job role: 380
done cf job role: 381
done cf job role: 382
done cf job role: 383
done cf job role: 384
done cf job role: 385
done cf job role: 386
done cf job role: 387
done cf job role: 388
done cf job role: 389
done cf job role: 390
done cf job role: 391
done cf job role: 392
done cf job role: 393
done cf job role: 394
done cf job role: 395
done cf job role: 396
done cf job role: 397
done cf job role: 398
done cf job role: 399
done cf job role: 400
done cf job role: 401
done cf job role: 402
done cf job role: 403
done cf job role: 404
done cf job role: 405
done cf job role: 406
done cf job role: 407
done cf job role: 408
done cf job role: 409
done cf job role: 410
done cf job role: 411
done cf job role: 412
done cf job role: 413
done cf job role: 414
done cf job role: 415
done cf job role: 416
done cf job role: 417
done cf job role: 418
done cf job role: 419
done cf job role: 420
done cf job role: 421
done cf job role: 422
done cf jo

done cf job role: 751
done cf job role: 752
done cf job role: 753
done cf job role: 754
done cf job role: 755
done cf job role: 756
done cf job role: 757
done cf job role: 758
done cf job role: 759
done cf job role: 760
done cf job role: 761
done cf job role: 762
done cf job role: 763
done cf job role: 764
done cf job role: 765
done cf job role: 766
done cf job role: 767
done cf job role: 768
done cf job role: 769
done cf job role: 770
done cf job role: 771
done cf job role: 772
done cf job role: 773
done cf job role: 774
done cf job role: 775
done cf job role: 776
done cf job role: 777
done cf job role: 778
done cf job role: 779
done cf job role: 780
done cf job role: 781
done cf job role: 782
done cf job role: 783
done cf job role: 784
done cf job role: 785
done cf job role: 786
done cf job role: 787
done cf job role: 788
done cf job role: 789
done cf job role: 790
done cf job role: 791
done cf job role: 792
done cf job role: 793
done cf job role: 794
done cf job role: 795
done cf jo

done cf job role: 1119
done cf job role: 1120
done cf job role: 1121
done cf job role: 1122
done cf job role: 1123
done cf job role: 1124
done cf job role: 1125
done cf job role: 1126
done cf job role: 1127
done cf job role: 1128
done cf job role: 1129
done cf job role: 1130
done cf job role: 1131
done cf job role: 1132
done cf job role: 1133
done cf job role: 1134
done cf job role: 1135
done cf job role: 1136
done cf job role: 1137
done cf job role: 1138
done cf job role: 1139
done cf job role: 1140
done cf job role: 1141
done cf job role: 1142
done cf job role: 1143
done cf job role: 1144
done cf job role: 1145
done cf job role: 1146
done cf job role: 1147
done cf job role: 1148
done cf job role: 1149
done cf job role: 1150
done cf job role: 1151
done cf job role: 1152
done cf job role: 1153
done cf job role: 1154
done cf job role: 1155
done cf job role: 1156
done cf job role: 1157
done cf job role: 1158
done cf job role: 1159
done cf job role: 1160
done cf job role: 1161
done cf job

In [282]:
len(scores_main)

1300

In [10]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_acc['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_acc['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_acc['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(acc_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_2_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_3_score})
df_acc.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Accounts Executive,['Restructuring and Insolvency Partner/Restruc...,0.967528,"['Restructuring and Insolvency Manager', 'Rest...",0.965312,['Restructuring and Insolvency Associate/Restr...,0.965312
1,Accounta Executive,['Restructuring and Insolvency Partner/Restruc...,0.980529,['Restructuring and Insolvency Senior/Restruct...,0.979568,"['Restructuring and Insolvency Manager', 'Rest...",0.979115
2,Accountant #SGUnitedJobs,['Restructuring and Insolvency Partner/Restruc...,0.974933,['Restructuring and Insolvency Senior/Restruct...,0.972805,"['Restructuring and Insolvency Manager', 'Rest...",0.971781
3,SENIOR INTERNAL AUDITOR (Senior Manager),"['Audit Partner/Audit Director', 'Audit Partne...",0.974330,"['Chief Financial Officer', 'Chief Financial O...",0.974210,['Mergers and Acquisitions Partner/Mergers and...,0.974204
4,Senior Accounts Executive,['Restructuring and Insolvency Partner/Restruc...,0.975611,"['Chief Financial Officer', 'Chief Financial O...",0.973484,['Restructuring and Insolvency Senior/Restruct...,0.972915
5,Finance Analyst,['Mergers and Acquisitions Associate/Mergers a...,0.978917,['Mergers and Acquisitions Senior/Mergers and ...,0.977790,['Restructuring and Insolvency Partner/Restruc...,0.976486
6,Finance and Admin Executive,['Restructuring and Insolvency Partner/Restruc...,0.981455,['Mergers and Acquisitions Senior/Mergers and ...,0.980297,['Restructuring and Insolvency Senior/Restruct...,0.980206
7,ACCOUNTANT,['Restructuring and Insolvency Partner/Restruc...,0.980687,"['Chief Financial Officer', 'Chief Financial O...",0.979989,['Mergers and Acquisitions Senior/Mergers and ...,0.979597
8,Income Auditor,"['Chief Financial Officer', 'Chief Financial O...",0.975973,['Restructuring and Insolvency Partner/Restruc...,0.974567,['Management Accountant/Financial Planning and...,0.974127
9,Corporate Finance Manager #SGUnitedJobs,['Mergers and Acquisitions Associate/Mergers a...,0.991608,['Mergers and Acquisitions Senior/Mergers and ...,0.990803,['Restructuring and Insolvency Partner/Restruc...,0.990795


In [11]:
df_acc.to_csv('accounting.csv')

# Education

In [116]:
edu = [['Education and Training']]
edu_dataframes = []

for i in edu:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    edu_dataframes.append(df)

edu_df = pd.concat(edu_dataframes)
edu_df.reset_index(inplace=True)

sfw_sector_edu = SFW[SFW['Sector'] == "Training and Adult Education"]
sfw_sector_edu.reset_index(inplace=True)

print(len(edu_df))
print(len(sfw_sector_edu))

942
13


In [117]:
edu_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(edu_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(edu_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_edu)):
        sfw = nlp(str(sfw_sector_edu['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_edu['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

done cf job role: 378
done cf job role: 379
done cf job role: 380
done cf job role: 381
done cf job role: 382
done cf job role: 383
done cf job role: 384
done cf job role: 385
done cf job role: 386
done cf job role: 387
done cf job role: 388
done cf job role: 389
done cf job role: 390
done cf job role: 391
done cf job role: 392
done cf job role: 393
done cf job role: 394
done cf job role: 395
done cf job role: 396
done cf job role: 397
done cf job role: 398
done cf job role: 399
done cf job role: 400
done cf job role: 401
done cf job role: 402
done cf job role: 403
done cf job role: 404
done cf job role: 405
done cf job role: 406
done cf job role: 407
done cf job role: 408
done cf job role: 409
done cf job role: 410
done cf job role: 411
done cf job role: 412
done cf job role: 413
done cf job role: 414
done cf job role: 415
done cf job role: 416
done cf job role: 417
done cf job role: 418
done cf job role: 419
done cf job role: 420
done cf job role: 421
done cf job role: 422
done cf jo

done cf job role: 751
done cf job role: 752
done cf job role: 753
done cf job role: 754
done cf job role: 755
done cf job role: 756
done cf job role: 757
done cf job role: 758
done cf job role: 759
done cf job role: 760
done cf job role: 761
done cf job role: 762
done cf job role: 763
done cf job role: 764
done cf job role: 765
done cf job role: 766
done cf job role: 767
done cf job role: 768
done cf job role: 769
done cf job role: 770
done cf job role: 771
done cf job role: 772
done cf job role: 773
done cf job role: 774
done cf job role: 775
done cf job role: 776
done cf job role: 777
done cf job role: 778
done cf job role: 779
done cf job role: 780
done cf job role: 781
done cf job role: 782
done cf job role: 783
done cf job role: 784
done cf job role: 785
done cf job role: 786
done cf job role: 787
done cf job role: 788
done cf job role: 789
done cf job role: 790
done cf job role: 791
done cf job role: 792
done cf job role: 793
done cf job role: 794
done cf job role: 795
done cf jo

In [119]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_edu['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_edu['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_edu['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(edu_df['Job Title'][i])

df_edu = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_edu.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Admin Support (School),"['Learning Systems Manager', 'Learning Systems...",0.934904,"['Learning and Operations Manager', 'Learning ...",0.934904,"['Learning Technology Designer', 'Learning Tec...",0.934904
1,Childcare Teacher,"['Learning Consultant/Learning Solutionist', '...",0.970053,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.970053,"['Learning Facilitator', 'Learning Facilitator...",0.970053
2,Academic Coordinator,"['Learning Systems Manager', 'Learning Systems...",0.965781,"['Learning and Operations Manager', 'Learning ...",0.965781,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.965781
3,Child-care Teacher,"['Learning Consultant/Learning Solutionist', '...",0.953383,"['Learning Facilitator', 'Learning Facilitator...",0.953383,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.953383
4,Preschool Teacher,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.967870,"['Learning Consultant/Learning Solutionist', '...",0.967870,"['Courseware Developer', 'Courseware Developer...",0.967870
5,PE Assistant,"['Learning Consultant/Learning Solutionist', '...",0.949538,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.949538,"['Learning and Operations Manager', 'Learning ...",0.949538
6,Experienced Language Teacher,"['Learning Systems Manager', 'Learning Systems...",0.943755,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.943755,"['Learning Consultant/Learning Solutionist', '...",0.943755
7,Life Coach (Student Care Centre),"['Learning Systems Manager', 'Learning Systems...",0.950833,"['Learning and Operations Manager', 'Learning ...",0.950833,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.950833
8,Chinese Language Assistant Teacher (华文助理老师),"['Learning Consultant/Learning Solutionist', '...",0.954920,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.954920,"['Courseware Developer', 'Courseware Developer...",0.954920
9,Jazz Dance teacher,"['Learning Systems Manager', 'Learning Systems...",0.947972,"['Learning and Operations Manager', 'Learning ...",0.947972,"['Curriculum Lead', 'Curriculum Lead', 'Curric...",0.947972


In [120]:
df_edu.to_csv('education.csv')

# Engine

In [19]:
engine = [['Engineering']]
engine_dataframes = []

for i in engine:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    engine_dataframes.append(df)

engine_df = pd.concat(engine_dataframes)
engine_df.reset_index(inplace=True)

sfw_sector_engine = SFW[SFW['Sector'] == "Engineering Services"]
sfw_sector_engine.reset_index(inplace=True)

print(len(engine_df))
print(len(sfw_sector_engine))

2282
29


In [20]:
engine_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(engine_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(engine_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_engine)):
        sfw = nlp(sfw_sector_engine['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_engine['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

done cf job role: 378
done cf job role: 379
done cf job role: 380
done cf job role: 381
done cf job role: 382
done cf job role: 383
done cf job role: 384
done cf job role: 385
done cf job role: 386
done cf job role: 387
done cf job role: 388
done cf job role: 389
done cf job role: 390
done cf job role: 391
done cf job role: 392
done cf job role: 393
done cf job role: 394
done cf job role: 395
done cf job role: 396
done cf job role: 397
done cf job role: 398
done cf job role: 399
done cf job role: 400
done cf job role: 401
done cf job role: 402
done cf job role: 403
done cf job role: 404
done cf job role: 405
done cf job role: 406
done cf job role: 407
done cf job role: 408
done cf job role: 409
done cf job role: 410
done cf job role: 411
done cf job role: 412
done cf job role: 413
done cf job role: 414
done cf job role: 415
done cf job role: 416
done cf job role: 417
done cf job role: 418
done cf job role: 419
done cf job role: 420
done cf job role: 421
done cf job role: 422
done cf jo

done cf job role: 751
done cf job role: 752
done cf job role: 753
done cf job role: 754
done cf job role: 755
done cf job role: 756
done cf job role: 757
done cf job role: 758
done cf job role: 759
done cf job role: 760
done cf job role: 761
done cf job role: 762
done cf job role: 763
done cf job role: 764
done cf job role: 765
done cf job role: 766
done cf job role: 767
done cf job role: 768
done cf job role: 769
done cf job role: 770
done cf job role: 771
done cf job role: 772
done cf job role: 773
done cf job role: 774
done cf job role: 775
done cf job role: 776
done cf job role: 777
done cf job role: 778
done cf job role: 779
done cf job role: 780
done cf job role: 781
done cf job role: 782
done cf job role: 783
done cf job role: 784
done cf job role: 785
done cf job role: 786
done cf job role: 787
done cf job role: 788
done cf job role: 789
done cf job role: 790
done cf job role: 791
done cf job role: 792
done cf job role: 793
done cf job role: 794
done cf job role: 795
done cf jo

done cf job role: 1119
done cf job role: 1120
done cf job role: 1121
done cf job role: 1122
done cf job role: 1123
done cf job role: 1124
done cf job role: 1125
done cf job role: 1126
done cf job role: 1127
done cf job role: 1128
done cf job role: 1129
done cf job role: 1130
done cf job role: 1131
done cf job role: 1132
done cf job role: 1133
done cf job role: 1134
done cf job role: 1135
done cf job role: 1136
done cf job role: 1137
done cf job role: 1138
done cf job role: 1139
done cf job role: 1140
done cf job role: 1141
done cf job role: 1142
done cf job role: 1143
done cf job role: 1144
done cf job role: 1145
done cf job role: 1146
done cf job role: 1147
done cf job role: 1148
done cf job role: 1149
done cf job role: 1150
done cf job role: 1151
done cf job role: 1152
done cf job role: 1153
done cf job role: 1154
done cf job role: 1155
done cf job role: 1156
done cf job role: 1157
done cf job role: 1158
done cf job role: 1159
done cf job role: 1160
done cf job role: 1161
done cf job

done cf job role: 1476
done cf job role: 1477
done cf job role: 1478
done cf job role: 1479
done cf job role: 1480
done cf job role: 1481
done cf job role: 1482
done cf job role: 1483
done cf job role: 1484
done cf job role: 1485
done cf job role: 1486
done cf job role: 1487
done cf job role: 1488
done cf job role: 1489
done cf job role: 1490
done cf job role: 1491
done cf job role: 1492
done cf job role: 1493
done cf job role: 1494
done cf job role: 1495
done cf job role: 1496
done cf job role: 1497
done cf job role: 1498
done cf job role: 1499
done cf job role: 1500
done cf job role: 1501
done cf job role: 1502
done cf job role: 1503
done cf job role: 1504
done cf job role: 1505
done cf job role: 1506
done cf job role: 1507
done cf job role: 1508
done cf job role: 1509
done cf job role: 1510
done cf job role: 1511
done cf job role: 1512
done cf job role: 1513
done cf job role: 1514
done cf job role: 1515
done cf job role: 1516
done cf job role: 1517
done cf job role: 1518
done cf job

done cf job role: 1833
done cf job role: 1834
done cf job role: 1835
done cf job role: 1836
done cf job role: 1837
done cf job role: 1838
done cf job role: 1839
done cf job role: 1840
done cf job role: 1841
done cf job role: 1842
done cf job role: 1843
done cf job role: 1844
done cf job role: 1845
done cf job role: 1846
done cf job role: 1847
done cf job role: 1848
done cf job role: 1849
done cf job role: 1850
done cf job role: 1851
done cf job role: 1852
done cf job role: 1853
done cf job role: 1854
done cf job role: 1855
done cf job role: 1856
done cf job role: 1857
done cf job role: 1858
done cf job role: 1859
done cf job role: 1860
done cf job role: 1861
done cf job role: 1862
done cf job role: 1863
done cf job role: 1864
done cf job role: 1865
done cf job role: 1866
done cf job role: 1867
done cf job role: 1868
done cf job role: 1869
done cf job role: 1870
done cf job role: 1871
done cf job role: 1872
done cf job role: 1873
done cf job role: 1874
done cf job role: 1875
done cf job

done cf job role: 2190
done cf job role: 2191
done cf job role: 2192
done cf job role: 2193
done cf job role: 2194
done cf job role: 2195
done cf job role: 2196
done cf job role: 2197
done cf job role: 2198
done cf job role: 2199
done cf job role: 2200
done cf job role: 2201
done cf job role: 2202
done cf job role: 2203
done cf job role: 2204
done cf job role: 2205
done cf job role: 2206
done cf job role: 2207
done cf job role: 2208
done cf job role: 2209
done cf job role: 2210
done cf job role: 2211
done cf job role: 2212
done cf job role: 2213
done cf job role: 2214
done cf job role: 2215
done cf job role: 2216
done cf job role: 2217
done cf job role: 2218
done cf job role: 2219
done cf job role: 2220
done cf job role: 2221
done cf job role: 2222
done cf job role: 2223
done cf job role: 2224
done cf job role: 2225
done cf job role: 2226
done cf job role: 2227
done cf job role: 2228
done cf job role: 2229
done cf job role: 2230
done cf job role: 2231
done cf job role: 2232
done cf job

In [22]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_engine['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_engine['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_engine['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(engine_df['Job Title'][i])

df_engine = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_engine.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Site Engineer,"['Manager (Engineering Procurement)', 'Manager...",0.956556,['Principal Engineer/Manager (Engineering Cons...,0.956556,"['Director (Project Financing)', 'Director (Pr...",0.956556
1,Palm Oleo Chemical Technical Advisor,['Principal Engineer/Manager (Engineering Cons...,0.948544,['Senior Engineer/Assistant Manager (Engineeri...,0.948544,['Principal Engineer/Manager (Engineering Desi...,0.948544
2,"VP, Data Science/Algorithm","['Director (Project Financing)', 'Director (Pr...",0.923544,['Chief Executive Officer/Managing Director/Ge...,0.923544,['Principal Engineer/Manager (Engineering Cons...,0.923544
3,Assistant Engineer (Electrical / Mechanical),['Principal Engineer/Manager (Engineering Cons...,0.941644,"['Director (Project Financing)', 'Director (Pr...",0.941644,"['Manager (Engineering Procurement)', 'Manager...",0.941644
4,Front-end engineer,"['Director (Project Financing)', 'Director (Pr...",0.903257,['Principal Engineer/Manager (Engineering Cons...,0.903257,['Chief Executive Officer/Managing Director/Ge...,0.903257
5,AV - System Designer,['Principal Engineer/Manager (Engineering Cons...,0.949753,['Senior Engineer/Assistant Manager (Engineeri...,0.949753,['Engineer/Supervisor (Engineering Constructio...,0.949753
6,Design Engineer,['Principal Engineer/Manager (Engineering Cons...,0.943046,"['Director (Project Financing)', 'Director (Pr...",0.943046,"['Manager (Engineering Procurement)', 'Manager...",0.943046
7,#SGUnitedJobs Electrical & Instrumentation Tec...,['Principal Engineer/Manager (Engineering Cons...,0.954491,['Senior Engineer/Assistant Manager (Engineeri...,0.954491,['Senior Engineer/Assistant Manager (Operation...,0.954491
8,3453 - Electronics Hardware R&D Manager [ Ele...,['Principal Engineer/Manager (Engineering Cons...,0.933633,['Senior Engineer/Assistant Manager (Engineeri...,0.933633,['Principal Engineer/Manager (Engineering Desi...,0.933633
9,Assembly Technician,['Principal Engineer/Manager (Engineering Cons...,0.938827,['Senior Engineer/Assistant Manager (Engineeri...,0.938827,['Engineer/Supervisor (Engineering Constructio...,0.938827


In [23]:
df_engine.to_csv('engineering.csv')

# Tourism

In [14]:
trav = [['Travel / Tourism']]
trav_dataframes = []

for i in trav:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    trav_dataframes.append(df)

travel_df = pd.concat(trav_dataframes)
travel_df.reset_index(inplace=True)

sfw_sector_trav = SFW[SFW['Sector'] == "Tourism"]
sfw_sector_trav.reset_index(inplace=True)

print(len(travel_df))
print(len(sfw_sector_trav))

11
50


In [15]:
travel_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(travel_df)):
    scores = []
    sfw_jobs = [] #create a list of scores for each cfw job (size of total no. of sfw jobs)
    cf = nlp(str(travel_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_trav)):
        sfw = nlp(str(sfw_sector_trav['Skill Title'][j]))
        scores.append(sfw.similarity(cf)) #each cfw job will have a list 
        sfw_jobs.append(sfw_sector_trav['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10


In [17]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_trav['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_trav['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_trav['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(travel_df['Job Title'][i])

df_tourism = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_tourism.head(50)

IndexError: list index out of range

In [128]:
df_tourism.to_csv('tourism.csv')

# Media

In [131]:
adv = [['Advertising / Media'], ['Entertainment']]
adv_dataframes = []

for i in adv:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    adv_dataframes.append(df)

media_df = pd.concat(adv_dataframes)
media_df.reset_index(inplace=True)

sfw_sector_media = SFW[SFW['Sector'] == "Media"]
sfw_sector_media.reset_index(inplace=True)

print(len(media_df))
print(len(sfw_sector_media))

306
85


In [132]:
media_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(media_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(media_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_media)):
        sfw = nlp(str(sfw_sector_media['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_media['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

In [133]:
media_df

,level_0,index,Unnamed: 0,clean_jd,Industry,Job Title,Skills,skills_replaced,Industry_nolist
0,0,48,62,job description responsibilities responsible ...,['Advertising / Media'],Content Cretion,"['advertising', 'blogging', 'content strategy'...","['advertising', 'copy editing', 'content strat...",Advertising / Media
1,1,93,116,provide inspired leadership for the organizat...,['Advertising / Media'],operation,"['business development', 'business process imp...","['business development', 'business analysis', ...",Advertising / Media
2,2,95,120,the senior copywriter is a copywriter who aft...,['Advertising / Media'],Senior Copywriter,"['advertising', 'copywriting', 'creative direc...","['advertising', 'copy editing', 'digital produ...",Advertising / Media
3,3,177,242,this is an exciting time to join dentsu aegis...,['Advertising / Media'],"International Client Partner, Mobile Apps","['account management', 'client presentation', ...","['account management', 'client presentation', ...",Advertising / Media
4,4,195,269,this is an exciting time to join dentsu aegis...,['Advertising / Media'],"Digital Performance Director, Mobile Apps","['client communication', 'digital media', 'dig...","['client communication', 'digital media', 'dig...",Advertising / Media
...,...,...,...,...,...,...,...,...,...
301,22,19911,28616,position summary animators are responsible fo...,['Entertainment'],Animator,"['3d', 'animation', 'character animation', 'co...","['3d', '3d animation', '3d animation', 'commun...",Entertainment
302,23,20902,30037,position summary the production manager repor...,['Entertainment'],Production Manager,"['collaboration', 'communication skills', 'con...","['collaboration', 'communication skills', 'con...",Entertainment
303,24,21037,30234,position summary senior modelers are responsi...,['Entertainment'],Senior Modeler,"['graphics', 'hard surface modeling', 'leaders...","['graphics', 'hard surface modeling', 'leaders...",Entertainment
304,25,23926,34405,provide service to all customers according to...,['Entertainment'],Service Crew - Part time / Full Time,"['customer satisfaction', 'customer service', ...","['customer service', 'customer service', 'lead...",Entertainment


In [134]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_media['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_media['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_media['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(media_df['Job Title'][i])

df_media = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_media.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Content Cretion,"['Studio Director/Outside Broadcast Director',...",0.977734,"['Executive Producer - Broadcast', 'Executive ...",0.977734,"['Post-Production Supervisor', 'Post-Productio...",0.977734
1,operation,"['Executive Producer - Broadcast', 'Executive ...",0.981942,"['Producer - Broadcast* (Casting)', 'Producer ...",0.981942,"['Assistant Producer - Broadcast* (Casting)', ...",0.981942
2,Senior Copywriter,"['Studio Director/Outside Broadcast Director',...",0.976182,"['Executive Producer - Broadcast', 'Executive ...",0.976182,"['Post-Production Supervisor', 'Post-Productio...",0.976182
3,"International Client Partner, Mobile Apps","['Executive Producer - Broadcast', 'Executive ...",0.980843,"['Community Development Specialist', 'Communit...",0.980843,"['Executive Producer - Film', 'Executive Produ...",0.980843
4,"Digital Performance Director, Mobile Apps","['Executive Producer - Film', 'Executive Produ...",0.961986,"['Executive Producer - Broadcast', 'Executive ...",0.961986,"['Community Development Specialist', 'Communit...",0.961986
5,"Head of Client Operations, Global","['Head - Programme Planning and Scheduling', '...",0.983473,"['Community Development Specialist', 'Communit...",0.983473,"['Community Development Executive', 'Community...",0.983473
6,Digital Marketing Executive,"['Community Development Specialist', 'Communit...",0.978960,"['Executive Producer - Broadcast', 'Executive ...",0.978960,"['Executive Producer - Film', 'Executive Produ...",0.978960
7,Digital Marketing Executive,"['Community Development Specialist', 'Communit...",0.978960,"['Executive Producer - Broadcast', 'Executive ...",0.978960,"['Executive Producer - Film', 'Executive Produ...",0.978960
8,Head of Sales,"['Community Development Specialist', 'Communit...",0.986066,"['Head of Marketing/Marketing Director', 'Head...",0.986066,"['Community Development Executive', 'Community...",0.986066
9,Communications Executive,"['Executive Producer - Broadcast', 'Executive ...",0.979779,"['Community Development Specialist', 'Communit...",0.979779,"['Head - Content', 'Head - Content', 'Head - C...",0.979779


In [135]:
df_media.to_csv('media.csv')

# Healthcare

In [25]:
health = [['Healthcare / Pharmaceutical'], ['Personal Care / Beauty'], ['Medical / Therapy Services']]
health_dataframes = []

for i in health:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    health_dataframes.append(df)

health_df = pd.concat(health_dataframes)
health_df.reset_index(inplace=True)

sfw_sector_health = SFW[SFW['Sector'] == "Healthcare"]
sfw_sector_health.reset_index(inplace=True)

print(len(health_df))
print(len(sfw_sector_health))

1026
43


In [26]:
health_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(health_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(health_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_health)):
        sfw = nlp(sfw_sector_health['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_health['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

done cf job role: 378
done cf job role: 379
done cf job role: 380
done cf job role: 381
done cf job role: 382
done cf job role: 383
done cf job role: 384
done cf job role: 385
done cf job role: 386
done cf job role: 387
done cf job role: 388
done cf job role: 389
done cf job role: 390
done cf job role: 391
done cf job role: 392
done cf job role: 393
done cf job role: 394
done cf job role: 395
done cf job role: 396
done cf job role: 397
done cf job role: 398
done cf job role: 399
done cf job role: 400
done cf job role: 401
done cf job role: 402
done cf job role: 403
done cf job role: 404
done cf job role: 405
done cf job role: 406
done cf job role: 407
done cf job role: 408
done cf job role: 409
done cf job role: 410
done cf job role: 411
done cf job role: 412
done cf job role: 413
done cf job role: 414
done cf job role: 415
done cf job role: 416
done cf job role: 417
done cf job role: 418
done cf job role: 419
done cf job role: 420
done cf job role: 421
done cf job role: 422
done cf jo

done cf job role: 751
done cf job role: 752
done cf job role: 753
done cf job role: 754
done cf job role: 755
done cf job role: 756
done cf job role: 757
done cf job role: 758
done cf job role: 759
done cf job role: 760
done cf job role: 761
done cf job role: 762
done cf job role: 763
done cf job role: 764
done cf job role: 765
done cf job role: 766
done cf job role: 767
done cf job role: 768
done cf job role: 769
done cf job role: 770
done cf job role: 771
done cf job role: 772
done cf job role: 773
done cf job role: 774
done cf job role: 775
done cf job role: 776
done cf job role: 777
done cf job role: 778
done cf job role: 779
done cf job role: 780
done cf job role: 781
done cf job role: 782
done cf job role: 783
done cf job role: 784
done cf job role: 785
done cf job role: 786
done cf job role: 787
done cf job role: 788
done cf job role: 789
done cf job role: 790
done cf job role: 791
done cf job role: 792
done cf job role: 793
done cf job role: 794
done cf job role: 795
done cf jo

In [27]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_health['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_health['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_health['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(health_df['Job Title'][i])

df_health = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_health.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Medical Sales Consultant / Senior Medical Sale...,['Pharmacy Technician Executive (Patient Care ...,0.967404,['Pharmacy Technician Executive (Drug Compound...,0.967404,"['Principal Speech Therapy Manager', 'Principa...",0.967404
1,Clinical Project Manager,"['Principal Physiotherapy Educator', 'Principa...",0.980390,"['Principal Occupational Therapy Educator', 'P...",0.980390,"['Principal Physiotherapy Researcher', 'Princi...",0.980390
2,TCM Clinic Assistant,"['Principal Physiotherapy Educator', 'Principa...",0.966427,"['Principal Occupational Therapy Educator', 'P...",0.966427,"['Principal Speech Therapy Educator', 'Princip...",0.966427
3,Clinic Assistant [ Novena / 5.5 Days / GP Clin...,['Pharmacy Technician Executive (Patient Care ...,0.951939,"['Patient Service Executive', 'Patient Service...",0.951939,"['Patient Service Supervisor', 'Patient Servic...",0.951939
4,"Medical Educationalist, CenMED","['Principal Speech Therapy Educator', 'Princip...",0.975297,"['Principal Physiotherapy Educator', 'Principa...",0.975297,"['Principal Occupational Therapy Educator', 'P...",0.975297
5,"#SGUnitedjobs"" Clinic Assistants",['Pharmacy Technician Executive (Patient Care ...,0.972155,"['Pharmacy Technician', 'Pharmacy Technician',...",0.972155,"['Patient Service Assistant Supervisor', 'Pati...",0.972155
6,RESEARCH SCIENTIST – CELL THERAPY,"['Principal Speech Therapy Educator', 'Princip...",0.940393,"['Principal Physiotherapy Educator', 'Principa...",0.940393,"['Principal Occupational Therapy Educator', 'P...",0.940393
7,RESEARCH SCIENTIST – ANTIBODY ENGINEERING,"['Principal Speech Therapy Educator', 'Princip...",0.943677,"['Principal Physiotherapy Educator', 'Principa...",0.943677,"['Principal Occupational Therapy Educator', 'P...",0.943677
8,"Strategic Account Management Lead AH, ROPU SEASK","['Principal Physiotherapy Educator', 'Principa...",0.972117,"['Principal Occupational Therapy Educator', 'P...",0.972117,"['Principal Speech Therapy Educator', 'Princip...",0.972117
9,Full time Dental Assistant/ Nurse #SGUnitedJobs,['Pharmacy Technician Executive (Patient Care ...,0.942726,"['Principal Speech Therapy Manager', 'Principa...",0.942726,"['Patient Service Executive', 'Patient Service...",0.942726


In [28]:
df_health.to_csv('health.csv')

# Retail

In [136]:
retail = [['Sales / Retail']]
retail_dataframes = []

for i in retail:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    retail_dataframes.append(df)

retail_df = pd.concat(retail_dataframes)
retail_df.reset_index(inplace=True)

sfw_sector_retail = SFW[SFW['Sector'] == "Retail"]
sfw_sector_retail.reset_index(inplace=True)

print(len(retail_df))
print(len(sfw_sector_retail))

936
19


In [137]:
retail_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(retail_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(retail_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_retail)):
        sfw = nlp(str(sfw_sector_retail['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_retail['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

done cf job role: 378
done cf job role: 379
done cf job role: 380
done cf job role: 381
done cf job role: 382
done cf job role: 383
done cf job role: 384
done cf job role: 385
done cf job role: 386
done cf job role: 387
done cf job role: 388
done cf job role: 389
done cf job role: 390
done cf job role: 391
done cf job role: 392
done cf job role: 393
done cf job role: 394
done cf job role: 395
done cf job role: 396
done cf job role: 397
done cf job role: 398
done cf job role: 399
done cf job role: 400
done cf job role: 401
done cf job role: 402
done cf job role: 403
done cf job role: 404
done cf job role: 405
done cf job role: 406
done cf job role: 407
done cf job role: 408
done cf job role: 409
done cf job role: 410
done cf job role: 411
done cf job role: 412
done cf job role: 413
done cf job role: 414
done cf job role: 415
done cf job role: 416
done cf job role: 417
done cf job role: 418
done cf job role: 419
done cf job role: 420
done cf job role: 421
done cf job role: 422
done cf jo

done cf job role: 751
done cf job role: 752
done cf job role: 753
done cf job role: 754
done cf job role: 755
done cf job role: 756
done cf job role: 757
done cf job role: 758
done cf job role: 759
done cf job role: 760
done cf job role: 761
done cf job role: 762
done cf job role: 763
done cf job role: 764
done cf job role: 765
done cf job role: 766
done cf job role: 767
done cf job role: 768
done cf job role: 769
done cf job role: 770
done cf job role: 771
done cf job role: 772
done cf job role: 773
done cf job role: 774
done cf job role: 775
done cf job role: 776
done cf job role: 777
done cf job role: 778
done cf job role: 779
done cf job role: 780
done cf job role: 781
done cf job role: 782
done cf job role: 783
done cf job role: 784
done cf job role: 785
done cf job role: 786
done cf job role: 787
done cf job role: 788
done cf job role: 789
done cf job role: 790
done cf job role: 791
done cf job role: 792
done cf job role: 793
done cf job role: 794
done cf job role: 795
done cf jo

In [140]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_retail['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_retail['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_retail['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(retail_df['Job Title'][i])

df_retail = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_retail.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Key Account Manager,"['E-Commerce Executive', 'E-Commerce Director'...",0.986823,"['Brand Associate/Assistant', 'Brand Associate...",0.986823,"['Merchandising Director', 'Merchandising Dire...",0.986823
1,Sales and Marketing Executive,"['Merchandising Associate/Assistant', 'Merchan...",0.969110,"['Visual Merchandising Associate/Assistant', '...",0.969110,['Merchandising Executive/Category Executive/H...,0.969110
2,Sales / Indoor Sales,"['Sales Supervisor/Brand Supervisor', 'Sales S...",0.968165,"['Merchandising Associate/Assistant', 'Merchan...",0.968165,"['Visual Merchandising Associate/Assistant', '...",0.968165
3,Key Account Senior Executive - Marketplace,"['Visual Merchandiser', 'Visual Merchandiser',...",0.978231,"['Sales Supervisor/Brand Supervisor', 'Sales S...",0.978231,['Merchandising Manager/Category Manager/Priva...,0.978231
4,Senior Executive - Content Findability,"['Sales Supervisor/Brand Supervisor', 'Sales S...",0.954865,"['Visual Merchandiser', 'Visual Merchandiser',...",0.954865,"['Store Manager/Outlet Manager', 'Store Manage...",0.954865
5,Laundry Service Sales Manager [West /B2B/ Basi...,"['Merchandising Director', 'Merchandising Dire...",0.975979,['Merchandising Executive/Category Executive/H...,0.975979,"['Merchandising Associate/Assistant', 'Merchan...",0.975979
6,Retail Sales [ Computer Hardware / Bugis / Fun...,"['Sales Supervisor/Brand Supervisor', 'Sales S...",0.971398,"['Sales Associate/Brand Associate', 'Sales Ass...",0.971398,"['Merchandising Associate/Assistant', 'Merchan...",0.971398
7,Retail Assistant [ Gaming Products / West / 6 ...,"['Sales Supervisor/Brand Supervisor', 'Sales S...",0.963992,"['E-Commerce Associate/Assistant', 'E-Commerce...",0.963992,"['Sales Associate/Brand Associate', 'Sales Ass...",0.963992
8,"Part-time Sales Assistant, Giant IMM","['Sales Supervisor/Brand Supervisor', 'Sales S...",0.966237,"['Sales Associate/Brand Associate', 'Sales Ass...",0.966237,"['Merchandising Associate/Assistant', 'Merchan...",0.966237
9,Shop Sales Assistant,"['Sales Supervisor/Brand Supervisor', 'Sales S...",0.964490,"['Merchandising Associate/Assistant', 'Merchan...",0.964490,"['Visual Merchandising Associate/Assistant', '...",0.964490


In [141]:
df_retail.to_csv('retail.csv')

# Logistics

In [29]:
log = [['Logistics / Supply Chain']]
log_dataframes = []

for i in log:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    log_dataframes.append(df)

logistics_df = pd.concat(log_dataframes)
logistics_df.reset_index(inplace=True)

sfw_sector_log = SFW[SFW['Sector'] == "Logistics"]
sfw_sector_log.reset_index(inplace=True)

print(len(logistics_df))
print(len(sfw_sector_log))

1142
55


In [30]:
logs_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(logistics_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(logistics_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_log)):
        sfw = nlp(sfw_sector_log['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_log['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

done cf job role: 0
done cf job role: 1
done cf job role: 2
done cf job role: 3
done cf job role: 4
done cf job role: 5
done cf job role: 6
done cf job role: 7
done cf job role: 8
done cf job role: 9
done cf job role: 10
done cf job role: 11
done cf job role: 12
done cf job role: 13
done cf job role: 14
done cf job role: 15
done cf job role: 16
done cf job role: 17
done cf job role: 18
done cf job role: 19
done cf job role: 20
done cf job role: 21
done cf job role: 22
done cf job role: 23
done cf job role: 24
done cf job role: 25
done cf job role: 26
done cf job role: 27
done cf job role: 28
done cf job role: 29
done cf job role: 30
done cf job role: 31
done cf job role: 32
done cf job role: 33
done cf job role: 34
done cf job role: 35
done cf job role: 36
done cf job role: 37
done cf job role: 38
done cf job role: 39
done cf job role: 40
done cf job role: 41
done cf job role: 42
done cf job role: 43
done cf job role: 44
done cf job role: 45
done cf job role: 46
done cf job role: 47
do

done cf job role: 378
done cf job role: 379
done cf job role: 380
done cf job role: 381
done cf job role: 382
done cf job role: 383
done cf job role: 384
done cf job role: 385
done cf job role: 386
done cf job role: 387
done cf job role: 388
done cf job role: 389
done cf job role: 390
done cf job role: 391
done cf job role: 392
done cf job role: 393
done cf job role: 394
done cf job role: 395
done cf job role: 396
done cf job role: 397
done cf job role: 398
done cf job role: 399
done cf job role: 400
done cf job role: 401
done cf job role: 402
done cf job role: 403
done cf job role: 404
done cf job role: 405
done cf job role: 406
done cf job role: 407
done cf job role: 408
done cf job role: 409
done cf job role: 410
done cf job role: 411
done cf job role: 412
done cf job role: 413
done cf job role: 414
done cf job role: 415
done cf job role: 416
done cf job role: 417
done cf job role: 418
done cf job role: 419
done cf job role: 420
done cf job role: 421
done cf job role: 422
done cf jo

done cf job role: 751
done cf job role: 752
done cf job role: 753
done cf job role: 754
done cf job role: 755
done cf job role: 756
done cf job role: 757
done cf job role: 758
done cf job role: 759
done cf job role: 760
done cf job role: 761
done cf job role: 762
done cf job role: 763
done cf job role: 764
done cf job role: 765
done cf job role: 766
done cf job role: 767
done cf job role: 768
done cf job role: 769
done cf job role: 770
done cf job role: 771
done cf job role: 772
done cf job role: 773
done cf job role: 774
done cf job role: 775
done cf job role: 776
done cf job role: 777
done cf job role: 778
done cf job role: 779
done cf job role: 780
done cf job role: 781
done cf job role: 782
done cf job role: 783
done cf job role: 784
done cf job role: 785
done cf job role: 786
done cf job role: 787
done cf job role: 788
done cf job role: 789
done cf job role: 790
done cf job role: 791
done cf job role: 792
done cf job role: 793
done cf job role: 794
done cf job role: 795
done cf jo

done cf job role: 1119
done cf job role: 1120
done cf job role: 1121
done cf job role: 1122
done cf job role: 1123
done cf job role: 1124
done cf job role: 1125
done cf job role: 1126
done cf job role: 1127
done cf job role: 1128
done cf job role: 1129
done cf job role: 1130
done cf job role: 1131
done cf job role: 1132
done cf job role: 1133
done cf job role: 1134
done cf job role: 1135
done cf job role: 1136
done cf job role: 1137
done cf job role: 1138
done cf job role: 1139
done cf job role: 1140
done cf job role: 1141


In [31]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_log['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_log['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_log['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(logistics_df['Job Title'][i])

df_logistics = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_logistics.head(50)

,cf_jobs,closest_1_sfw_jobs,closest_1_score,closest_2_sfw_jobs,closest_2_score,closest_3_sfw_jobs,closest_3_score
0,Order Fullfillment Specialist,['Logistics Solutions and Implementation Direc...,0.981930,['IT Business Solutions Project Specialist/Dig...,0.981930,"['Logistics Solutions Manager', 'Logistics Sol...",0.981930
1,Delivery Driver,['Warehouse Operations Manager/Inventory Manag...,0.959170,['Import Export Manager/Freight Allocation Man...,0.959170,['Logistics Solutions Specialist/Logistics Sol...,0.959170
2,Delivery Driver cum Assistant #SGUnitedJobs,['Warehouse Operations Manager/Inventory Manag...,0.953357,['Warehouse Operations Executive/Inventory Man...,0.953357,['Logistics Solutions Specialist/Logistics Sol...,0.953357
3,Traffic Controller,['Import Export Manager/Freight Allocation Man...,0.968036,['Logistics Solutions and Implementation Direc...,0.968036,['IT Business Solutions Project Specialist/Dig...,0.968036
4,PRIME MOVER DRIVER,['Warehouse Operations Manager/Inventory Manag...,0.937518,['Logistics Solutions Specialist/Logistics Sol...,0.937518,"['Logistics Solutions Manager', 'Logistics Sol...",0.937518
5,Operations Executive,['Chief Executive Officer/Chief Operating Offi...,0.967088,['Business Development Director/Country Route ...,0.967088,['Logistics Solutions and Implementation Direc...,0.967088
6,Assistant NPI Planning & Artwork Manager,['Logistics Solutions and Implementation Direc...,0.964673,"['Logistics Solutions Manager', 'Logistics Sol...",0.964673,['Logistics Solutions Specialist/Logistics Sol...,0.964673
7,Warehouse Assistant,['Logistics Solutions and Implementation Direc...,0.965057,['Warehouse Operations Manager/Inventory Manag...,0.965057,['IT Business Solutions Project Specialist/Dig...,0.965057
8,PRIME MOVER DRIVER,['Warehouse Operations Manager/Inventory Manag...,0.937518,['Logistics Solutions Specialist/Logistics Sol...,0.937518,"['Logistics Solutions Manager', 'Logistics Sol...",0.937518
9,Class 3 Night Chiller Truck Delivery Driver,['Import Export Manager/Freight Allocation Man...,0.948825,['Logistics Solutions and Implementation Direc...,0.948825,['IT Business Solutions Project Specialist/Dig...,0.948825


In [32]:
df_logistics.to_csv('logistics.csv')